In [ ]:
# pip install --upgrade pip

In [ ]:
#  pip install kiwipiepy

In [ ]:
# pip install soynlp

In [1]:
import pandas as pd
import numpy as np

# 텍스트 파일로 받기

- 텍스트 유형
    - 댓글 : [ㅇㅇㅇㅇ] 식

In [2]:
df = pd.read_csv('df_contents.csv')
df.review.isnull().sum()
df = df.dropna()

In [3]:
df.groupby('title').count()

,Unnamed: 0,review
title,,
군검사 도베르만,44,44
그 해 우리는,15,15
그리드,15,15
그린마더스클럽,15,15
기상청 사람들: 사내연애 잔혹사,14,14
나의 아저씨,3886,3886
나의 해방일지,62,62
내일,42,42
눈이 부시게,1461,1461


In [4]:
sentences = df.review.to_list()
사내맞선 = df[df.title == '사내 맞선'].review.to_list()

In [ ]:
[re.sub(re_code, ' ', sent) for sent in sentences]

In [5]:
from kiwipiepy import Kiwi, Match
from kiwipiepy.utils import Stopwords
import re
text_list = sentences
re_code = '[^ ㄱ-ㅎ가-힣0-9]'
stopwords = Stopwords()
stopwords.add(('결과','NNG'))

# 논문 프로세스 그대로
### 1. 단어토큰
### 2. 불용어 제거
### 3. 명사 추출

https://velog.io/@mare-solis/LDA-%ED%86%A0%ED%94%BD-%EB%AA%A8%EB%8D%B8%EB%A7%81%EC%9C%BC%EB%A1%9C-%EC%BD%98%ED%85%90%EC%B8%A0-%EB%A6%AC%EB%B7%B0%EB%A5%BC-%EB%B6%84%EC%84%9D%ED%95%98%EC%9E%90
젠심 lda

In [ ]:
# 1,2 전체문장, 단어토큰, 불용어 제거
kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True)

token_sent = []
for sentence in 사내맞선:
    token = kiwi.tokenize(sentence, normalize_coda=True, stopwords=stopwords)
    temp = []
    for tok in token:
        if tok.tag in {'NNG','NNP'} and len(tok.form)>1:
            temp.append(tok.form)
    if temp:
        token_sent.append(temp)

token_sent[:3]

### LDA 토픽모델링

In [7]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
# 사전 생성
dictionary = corpora.Dictionary(token_sent)

# 빈도수, 비중 필터
dictionary.filter_extremes(no_below=2, no_above=0.5)

# count vectorize
corpus = [dictionary.doc2bow(text) for text in token_sent]

2022-04-15 17:38:43,031 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-04-15 17:38:43,039 : INFO : built Dictionary(1244 unique tokens: ['감상', '기대', '김세정', '만화', '박서준']...) from 388 documents (total 3499 corpus positions)
2022-04-15 17:38:43,042 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(1244 unique tokens: ['감상', '기대', '김세정', '만화', '박서준']...) from 388 documents (total 3499 corpus positions)", 'datetime': '2022-04-15T17:38:43.041410', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2022-04-15 17:38:43,043 : INFO : discarding 852 tokens: [('연인', 1), ('외모', 1), ('전편', 1), ('파리', 1), ('당황', 1), ('백김치', 1), ('치얼스', 1), ('효섭찡', 1), ('극단', 1), ('라스폰트리', 1)]...
2022-04-15 17:38:43,044 : INFO : keeping 392 tokens which were in no less than 2 and no more than 194 (=50.0%) documents
2022-04-15 17:38:43,045 : INFO : resulting dictionary: Dictiona

### lda 모델링
class gensim.models.ldamodel.LdaModel(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)
- num_topics: 생성될 토픽의 개수
- chunksize: 한번의 트레이닝에 처리될 문서의 개수
- passes: 전체 코퍼스 트레이닝 횟수
- interations: 문서 당 반복 횟수

- Cogerence는 토픽일관성, eval_every=1로 설정하면 일관성 평가가능 but 개느림

In [ ]:
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [14]:
# 시각화
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis)


# 문장분리 활용 하기

In [16]:
def text_preprocess(text_list,re_code):

    # sent_tokenize by Kiwi
    kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True)

    sentences_list = []
    for sentence in text_list:
        sent_split = kiwi.split_into_sents(sentence)
        for sent in sent_split:
            sentences_list.append(re.sub(re_code, '', sent.text))
    return sentences_list, text_list

In [17]:
sentences_list, text_list = text_preprocess(text_list,re_code)
sentences_list[:10]

['이 화려한 캐스팅 라인업을 보고 안 볼수가 있나 ㄷㄷㄷ',
 '게다가 노희경 극본ㄷㄷㄷ',
 '바로 이거지',
 '눈과 귀를 홀리는 이 몰입도',
 '심지어 짠내나는 차승원이라니ㅋㅋ',
 '너무 좋다',
 '침묵의 순간들이 더욱 처연하게 빛나는 노희경 극본의 또 다른 경지',
 '세상에',
 '노희경 작가와 이 배우들이라면 어떤 스토리가 됐든 꼭 보고싶다',
 '하 진짜 좋아하는 배우 전부 다 나오네']

# soynlp 활용한 명사추출

In [18]:
# 1-2. Noun Extractor 통해 단어 코퍼스 추출
def Noun_ext(sentences_list, frequency = 3, score = 0.5):
    from soynlp.utils import DoublespaceLineCorpus
    from soynlp.noun import LRNounExtractor_v2
    
    noun_extractor = LRNounExtractor_v2(verbose=False)
    nouns = noun_extractor.train_extract(sentences_list)

        # 딕셔너리에서 valid한 키 값만 뽑기
    keys_list = list(nouns.keys())
    valid_key_dict = {}
    for key in keys_list:
        if float(nouns[key].frequency) > frequency and float(nouns[key].score) > score:
            valid_key_dict[key] = nouns[key]

    # valid 키값 리스트에 추가
    valid_key = list(valid_key_dict.keys())
    return valid_key


In [19]:
noun_ext = Noun_ext(sentences_list)

In [20]:
noun_ext

['신이말하는대로',
 '지금우리학교는',
 '하이퍼리얼리즘',
 '디어마이프렌즈',
 '수다블록버스터',
 '부산국제영화제',
 '2000년대',
 '미스터션샤인',
 '오리지널리티',
 '드라마드라마',
 '대사하나하나',
 '스테레오타입',
 '미스터선샤인',
 '백상예술대상',
 '아포칼립스물',
 '칠드런오브맨',
 '바람바람바람',
 '박동훈이선균',
 '이태원클라쓰',
 '디스토피아적',
 '이병헌감독님',
 '김혜자선생님',
 '나희도백이진',
 '2020년대',
 '연상호감독님',
 '디스토피아물',
 '2010년대',
 '이태원클라스',
 '외국인노동자',
 '오징어게임',
 '펜트하우스',
 '아포칼립스',
 '부부의세계',
 '인생드라마',
 '미니시리즈',
 '사랑이야기',
 '등장인물들',
 '한국드라마',
 '러브스토리',
 '성장드라마',
 '10화정도',
 '인생캐릭터',
 '카타르시스',
 '주말드라마',
 '클라이막스',
 '있어보이려',
 '필모그래피',
 '코로나19',
 '블록버스터',
 '나의아저씨',
 '발암캐릭터',
 '임대아파트',
 '조기축구회',
 '2021년',
 '클리셰범벅',
 '염세주의자',
 '프로파일링',
 '미스캐스팅',
 '애니메이션',
 '2009년',
 '씨유어게인',
 '2019년',
 '30대버전',
 '여성캐릭터',
 '페미니스트',
 '폭력적인것',
 '연기파배우',
 '오징어짬뽕',
 '시놉시스만',
 '멜로드라마',
 '라이어게임',
 '싸이코패스',
 '멜로가체질',
 '만들었으면',
 '보여주기식',
 '소시오패스',
 '더치페이스',
 '프로파일러',
 '이런드라마',
 '1516화',
 '디스토피아',
 '일제강점기',
 '다큐멘터리',
 '여자캐릭터',
 '영화드라마',
 '막장드라마',
 '개그포인트',
 '아이덴티티',
 '터미네이터',
 '피었습니다',
 '블랙코미디',
 '데스게임류',
 '크리스마스',
 '김은숙작가',
 '알츠하이머',
 '한소희배우',
 '남자주

# 추출한 명사목록 kiwi 단어장에 추가 후 형태소 분석
### 형태소만 남긴 문장으로 추후 토픽모델링할 것
    - seperate가 True일 경우 작품별 단어가 2차원 리스트로 저장 되어 있으므로 1차원 리스트만 뽑아서 학습시킬 것
    - kiwi 문장분리된 리스트 넣으면 안됨(너무 짧아서)
    - ,기준으로 split한 리스트 넣을 것

In [21]:
def lets_go(text_list, noun_ext):
    kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True)
    for i in range(len(noun_ext)):
        kiwi.add_user_word(noun_ext[i],'NNP')


# word tokenize and join


    sent_tokens = []
    for sent in text_list:
        temp = []
        for tok in kiwi.tokenize(sent):
            if tok.tag[0] in {'N','V','M'} and len(tok.form) > 1: 
                temp.append(tok.form) # 1글자 이상 단어만 추출
        if ' '.join(temp) != '':
            sent_tokens.append(' '.join(temp))
    return sent_tokens

In [22]:
sent_tokens = lets_go(text_list, noun_ext)

In [ ]:
# 영어는 kiwi에선 sl로 처리되어서 자동으로 짤림
kiwi = Kiwi()
kiwi.tokenize('every time i come around you never can\'t say no')

# BERTopic 테스트

In [ ]:
pip install bertopic[visualization]

In [ ]:
# pip install bertopic

In [26]:
from bertopic import BERTopic

ModuleNotFoundError: No module named 'bertopic'

In [27]:
from bertopic import BERTopic

model = BERTopic()

tf, p = model.fit_transform(sent_tokens) # 전처리 없이 그냥도 돌려보자

ModuleNotFoundError: No module named 'bertopic'

In [ ]:
for i in range(30):
    print(i+1, '번째 댓글 토픽 :', model.get_topic(i))

Bertopic 수찬님 방법

In [ ]:
#  pip install konlpy

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = [token.form for token in self.tagger.tokenize(sent)]
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [ ]:
kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True)
for i in range(len(noun_ext)):
    kiwi.add_user_word(noun_ext[i],'NNP')
custom_tokenizer = CustomTokenizer(kiwi)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                 vectorizer_model=vectorizer,
                 nr_topics=50,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(sent_tokens)

In [ ]:
for i in range(20):
    print(i+1, '번째 댓글 토픽 :', model.get_topic(i))

1 번째 댓글 토픽 : [('드라마', 0.024893714265391005), ('연기', 0.024746582715839384), ('유아인', 0.017236517835243974), ('배우', 0.01458429844067057), ('연기력', 0.013992654490463538), ('부산행', 0.013969998226091898), ('한국', 0.013394841389205683), ('연상호', 0.012900679418549867), ('영화', 0.012881056005122394), ('박정민', 0.012717487646495322)]
2 번째 댓글 토픽 : [('드라마', 0.03830841019863968), ('시즌', 0.026213234375470205), ('연출', 0.02200581085369505), ('연기', 0.020943536454314286), ('배우', 0.018430706820627644), ('연상호', 0.01745386038980276), ('랜덤', 0.017074123280987827), ('영화', 0.01638327860559346), ('유아인', 0.016181220824922914), ('화살촉', 0.01600422607541458)]
3 번째 댓글 토픽 : [('시즌', 0.050119966476868595), ('나오', 0.03640049836710617), ('필요', 0.03409862096672714), ('계절', 0.033043898360075104), ('파이트', 0.030577749195707223), ('노잼', 0.027126506179335205), ('우리', 0.02675332192267203), ('어찌', 0.025450230817657134), ('아니', 0.02418358706606285), ('아닌듯', 0.02394032270438201)]
4 번째 댓글 토픽 : [('배우들', 0.03203550871178342), ('정말', 0.0299

망했으니 tomotopy 해보겠습니다

In [ ]:
pip install tomotopy

In [ ]:
import tomotopy as tp
mdl = tp.LDAModel(k=20)
for line in sent_tokens:
    mdl.add_doc(line.strip().split())
 
for i in range(100):
    mdl.train()
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))
 
for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

Iteration: 0	Log-likelihood: -8.680819679745353
Iteration: 1	Log-likelihood: -8.449892747431015
Iteration: 2	Log-likelihood: -8.359928596764352
Iteration: 3	Log-likelihood: -8.306852997355756
Iteration: 4	Log-likelihood: -8.264342462896412
Iteration: 5	Log-likelihood: -8.242447145665434
Iteration: 6	Log-likelihood: -8.206298778056137
Iteration: 7	Log-likelihood: -8.184207444692507
Iteration: 8	Log-likelihood: -8.166062114960768
Iteration: 9	Log-likelihood: -8.141464817610427
Iteration: 10	Log-likelihood: -8.154815289007663
Iteration: 11	Log-likelihood: -8.134875064311238
Iteration: 12	Log-likelihood: -8.121495803714447
Iteration: 13	Log-likelihood: -8.111935152002047
Iteration: 14	Log-likelihood: -8.099988804061928
Iteration: 15	Log-likelihood: -8.100415356568023
Iteration: 16	Log-likelihood: -8.102715945647365
Iteration: 17	Log-likelihood: -8.081611758094628
Iteration: 18	Log-likelihood: -8.076450420261846
Iteration: 19	Log-likelihood: -8.069751784999886
Iteration: 20	Log-likelihood: 